In [ ]:
from __future__ import absolute_import ,division ,print_function
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pandas as pd

import ROOT

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Lambda,multiply,Multiply,RepeatVector,Flatten,Concatenate,Dropout
from tensorflow.keras.models import Model

import tensorflow.keras.backend as K
from functools import partial

In [ ]:
#K.set_session(tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=4)))

In [ ]:
n_particles = 100000/2
p = np.random.uniform(0.3,10.,int(n_particles))
mp = np.random.uniform(1/10.,1/0.3,int(n_particles))
p_mp = 1./mp 
p_ges = np.concatenate([p,p_mp])

In [ ]:

mass_e = 0.000511
mass_mu = 0.105
mass_pi = 0.139
mass_K = 0.494
mass_p = 0.938
masses = [ mass_e,mass_mu, mass_pi, mass_K , mass_p]
masses

In [ ]:
signals = []
for mass in masses:
    ITS_tmp = []
    TPCROC0_tmp = []
    TPCROC1_tmp = []
    TPCROC2_tmp = []
    TRD_tmp = []
    TOF_tmp = []
    BBS_tmp = []
    BBA_tmp = []
    beta_tmp = []
    pmeas_tmp = []
    for p in p_ges:
        bg = p/mass
        beta = bg/math.sqrt(1.+ bg*bg);
        BBS = ROOT.AliExternalTrackParam.BetheBlochSolid(bg)
        BBA = ROOT.AliExternalTrackParam.BetheBlochAleph(bg)
        ITS_tmp.append(np.random.normal(BBS,0.1*BBS) ) ## ITS dEdx = smeared gaus 10% 
        TPCROC0_tmp.append(np.random.normal(BBA,0.1*BBA) )## TPC dEdx = smeared gaus 10% for 1st layer
        TPCROC1_tmp.append(np.random.normal(BBA,0.1*BBA) )  ## TPC dEdx = smeared gaus 10% for 2nd layer
        TPCROC2_tmp.append(np.random.normal(BBA,0.1*BBA) )  ## TPC dEdx = smeared gaus 10% for 3d layer
        TRD_tmp.append(np.random.normal(BBA,0.1*BBA) )  ## TRD dEdx = smeared gaus 10% 
        TOF_tmp.append(np.random.normal(beta,0.01*beta) )  ## TOF - smeared with .... gaussian
        pmeas_tmp.append(np.random.normal(p,0.01*p))
        BBS_tmp.append(BBS)
        BBA_tmp.append(BBA)
        beta_tmp.append(beta)
        
    signals.append({'ITS': ITS_tmp, 'TPCROC0': TPCROC0_tmp, 'TPCROC1': TPCROC1_tmp, 'TPCROC1': TPCROC1_tmp, 
                    'TPCROC2': TPCROC2_tmp, 'TRD': TRD_tmp, 'TOF': TOF_tmp, 'BBS': BBS_tmp, 'BBA': BBA_tmp, "beta": beta_tmp,
                   "pmeas": pmeas_tmp})

In [ ]:
df_list=[]
for i, val in enumerate(masses):
    df = pd.DataFrame.from_dict(signals[i])
    df['p'] = pd.Series(p_ges, index=df.index)
    df['particle'] = pd.Series(i, index=df.index)
    df_list.append(df)
df_all = pd.concat([df_list[0],df_list[2],df_list[3],df_list[4]], ignore_index=True)

In [ ]:
m_inv  = np.array([1./mass_e,1./mass_mu,1./mass_pi,1./mass_K,1./mass_p])
bg     = np.log(pd.DataFrame(m_inv[df_all["particle"]]*df_all["p"]))
bg.columns=['lnbg']
df_all = pd.concat([df_all,bg], axis = 1).sample(frac=1)
df_all.head(10)

In [ ]:
plt.hist2d(df_all["p"], df_all["TPCROC1"], bins=(100, 100), cmap=plt.cm.jet, range = [[0.2, 2], [0.5, 3]])
plt.plot()

In [ ]:
plt.hist2d(df_all["p"], df_all["TOF"], bins=(100, 100), cmap=plt.cm.jet, range = [[0.2, 2], [0.0, 1.1]])
plt.plot()

In [ ]:
def BetheBlochAlephNP(lnbg,kp1=0.76176e-1,kp2=10.632,kp3=0.13279e-4,kp4=1.8631,kp5=1.9479):
    bg   = np.exp(lnbg)
    beta = bg/np.sqrt(1.+ bg*bg)
    aa   = np.exp(kp4*np.log(beta))
    bb   = np.exp(-kp5*np.log(bg))
    bb   = np.log(kp3+bb)
    return (kp2-aa-bb)*kp1/aa

def BetheBlochGeantNP(lnbg,kp0=2.33,kp1=0.20,kp2=3.00,kp3=173e-9,kp4=0.49848):
    bg   = np.exp(lnbg)
    mK  = 0.307075e-3
    me  = 0.511e-3
    rho = kp0
    x0  = kp1*2.303
    x1  = kp2*2.303
    mI  = kp3
    mZA = kp4
    bg2 = bg*bg
    maxT= 2*me*bg2
    
    d2=0.
    x=np.log(bg)
    lhwI=np.log(28.816*1e-9*np.sqrt(rho*mZA)/mI)
    if x>x1 :
        d2 = lhwI + x - 0.5
    else :
        if x>x0:
            r=(x1-x)/(x1-x0)
            d2 = lhwI + x - 0.5 + (0.5 - lhwI - x0)*r*r*r
        
    return mK*mZA*(1+bg2)/bg2*(0.5*np.log(2*me*bg2*maxT/(mI*mI)) - bg2/(1+bg2) - d2)

def BetheBlochSolidNP(lnbg):
    return BetheBlochGeantNP(lnbg)

In [ ]:

def BetheBlochAleph(lnbg,kp1=0.76176e-1,kp2=10.632,kp3=0.13279e-4,kp4=1.8631,kp5=1.9479):
    bg   = K.exp(lnbg)
    beta = bg/K.sqrt(1.+ bg*bg)
    aa   = K.exp(kp4*K.log(beta))
    bb   = K.exp(-kp5*K.log(bg))
    bb   = K.log(kp3+bb)
    return (kp2-aa-bb)*kp1/aa


def BetheBlochGeant(lnbg,kp0=2.33,kp1=0.20,kp2=3.00,kp3=173.0e-9,kp4=0.49848):
    bg=K.exp(lnbg)
    mK  = 0.307075e-3
    me  = 0.511e-3
    rho = kp0
    x0  = kp1*2.303
    x1  = kp2*2.303
    mI  = kp3
    mZA = kp4
    bg2 = bg*bg
    maxT= 2*me*bg2
    
    
    x=lnbg
    lhwI=K.log(28.816e-9*K.sqrt(K.cast(rho*mZA,dtype=float))/mI)

    d2=K.switch(K.greater(x,x1),lhwI + x - 0.5,
               K.switch(K.greater(x,x0),lhwI + x - 0.5 + (0.5 - lhwI - x0)*(((x1-x)/(x1-x0))**3),0.*bg))
        
    return mK*mZA*(1+bg2)/bg2*(0.5*K.log(2*me*bg2*maxT/(mI*mI)) - bg2/(1+bg2) - d2)

    
def BetheBlochSolid(lnbg):
    return BetheBlochGeant(lnbg)

def getbeta(lnbg):
    bg   = K.exp(lnbg)
    return bg/K.sqrt(1.+ bg*bg)


In [ ]:


def custom_loss(y_true, y_pred):

    d = (y_true - y_pred)/ y_true 
    d0=d[:,0]
    d1=d[:,1]*10
    d2=d[:,2]
    d3=d[:,3]
    d4=d[:,4]
    d5=d[:,5]
    d=d0*d0+d1*d1+d2*d2+d3*d3+d4*d4+d5*d5

    return d/6.

inputs = Input(shape=(7,))
enc    = Dense(units=64, activation='selu')(inputs)
enc    = Dense(units=64, activation='selu')(enc)
enc    = Dense(units=64, activation='selu')(enc)
#enc    = Dense(units=64, activation='selu')(enc)

gb     = Dense(units=1, activation='linear')(enc)

BBA    = Lambda(BetheBlochAleph)(gb)
BBA4   = RepeatVector(4)(BBA)
BBA4    = Flatten()(BBA4) 
BBS    = Lambda(BetheBlochSolid)(gb)
TOF    = Lambda(getbeta)(gb)
final  = Concatenate(axis=-1)([BBS,TOF,BBA4])

modelgb= Model(inputs=inputs,outputs=gb)
modelBB= Model(inputs=inputs,outputs=BBA)
modelBS= Model(inputs=inputs,outputs=BBS)
modelTF= Model(inputs=inputs,outputs=TOF)
modelfi= Model(inputs=inputs,outputs=final)

modelfi.compile(loss=custom_loss,optimizer='adam',metrics=['mse'])
modelgb.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [ ]:
modelfi.summary()

In [ ]:
modelgb.summary()

In [ ]:
modelBB.summary()

In [ ]:
train, test =train_test_split(df_all, test_size=0.5)
train_data = train[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]
test_data  = test[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]]

train_fdata= train[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD"]]
test_fdata = test[["ITS", "TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD"]]

train_gb   = train[["lnbg"]]
test_gb    = test[["lnbg"]]

In [ ]:
# take scaler from unmodified values ?
scaler = StandardScaler()
scaler.fit(train[["ITS","TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]])



In [ ]:
#modelgb.fit(scaler.transform(train_data),train_gb, epochs=10, batch_size=64, 
#          validation_data=[scaler.transform(test_data),test_gb])

In [ ]:
modelfi.fit(scaler.transform(train_data),train_fdata, epochs=12, batch_size=128, 
          validation_data=[scaler.transform(test_data),test_fdata])

In [ ]:
out=modelfi.predict(scaler.transform(test_data))
AE_predict = pd.DataFrame(out)
AE_predict.columns = ["ITS_ae", "TOF_ae", "TPCROC0_ae", "TPCROC1_ae", "TPCROC2_ae", "TRD_ae"]
ogb=(modelgb.predict(scaler.transform(test_data)))
GB_predict = pd.DataFrame(ogb)
GB_predict.columns = ["gb_ae"]
test = test.reset_index()
df_test = pd.concat([test,AE_predict,GB_predict], axis = 1)
df_test.head()

In [ ]:
plt.hist2d(df_test["p"], df_test["TPCROC1"], bins=(100, 100), cmap=plt.cm.jet, range = [[0.2, 2], [0.5, 3]])
plt.show()
plt.hist2d(df_test["p"], df_test["TPCROC1_ae"], bins=(100, 100), cmap=plt.cm.jet, range = [[0.2, 2], [0.5, 3]])
plt.show()


In [ ]:
particle_id = 2
delta=df_test.query("particle==" + str(particle_id))["TPCROC1_ae"]-df_test.query("particle==" + str(particle_id))["BBA"]
plt.hist2d(df_test.query("particle==" + str(particle_id))["p"],delta, bins=(50, 50), range = [[0.2, 2], [-0.2, 0.2]], cmap=plt.cm.BuPu)
plt.show()

In [ ]:
particle_id = 0
delta=df_test.query("particle==" + str(particle_id))["TPCROC1_ae"]-df_test.query("particle==" + str(particle_id))["BBA"]
plt.hist2d(df_test.query("particle==" + str(particle_id))["p"],delta, bins=(100, 100), range = [[0.2, 2], [-0.2, 0.2]])
plt.show()

In [ ]:
particle_id = 0
delta=df_test.query("particle==" + str(particle_id))["TPCROC1_ae"]-df_test.query("particle==" + str(particle_id))["BBA"]
plt.hist2d(df_test.query("particle==" + str(particle_id))["p"],delta, bins=(100, 100), range = [[0.2, 2], [-0.2, 0.2]])
plt.show()

In [ ]:
particle_id = 4
delta=df_test.query("particle==" + str(particle_id))["TPCROC1_ae"]-df_test.query("particle==" + str(particle_id))["BBA"]
momentum = df_test.query("particle==" + str(particle_id))["p"]
sns.jointplot(momentum, delta)
#plt.hist2d(momentum,delta, bins=(100, 100), range = [[0.2, 2], [-0.2, 0.2]], cmap=plt.cm.BuPu)
#plt.show()

In [ ]:
particle_id = 2
plt.hist2d(df_test.query("particle==" + str(particle_id))["p"], df_test.query("particle==" + str(particle_id))["TPCROC1"], bins=(100, 100), cmap=plt.cm.jet, range = [[0.2, 2], [0.5, 3]])
plt.show()
plt.hist2d(df_test.query("particle==" + str(particle_id))["p"], df_test.query("particle==" + str(particle_id))["TPCROC1_ae"], bins=(100, 100), cmap=plt.cm.jet, range = [[0.2, 2], [0.5, 3]])
plt.show()


In [ ]:
#plt.yscale('log')
plt.axis([0.,10.,0.,10.])
plt.scatter(df_test["p"],df_test["gb_ae"],c=df_test["particle"])
plt.show()

In [ ]:
#plt.yscale('log')
plt.axis([0.,10.,0.,10.])
plt.scatter(df_test["p"],np.exp(df_test["gb_ae"]),c=df_test["particle"])
plt.show()

In [ ]:

plt.hist(df_test["p"]/np.exp(df_test["gb_ae"]),bins=100, range=[0,1.5])
plt.xscale('log')

In [ ]:
particle_id = 3
plt.hist(df_test.query("particle==" + str(particle_id))["p"]/np.exp(df_test.query("particle==" + str(particle_id))["gb_ae"]),bins=100, range=[0,1.5])
plt.xscale('log')

In [ ]:
particle_id = 4
plt.hist(df_test.query("particle==" + str(particle_id)+ "and p>0.2 and p < 3.")["p"]/np.exp(df_test.query("particle==" + str(particle_id)+ "and p>0.2 and p < 3.")["gb_ae"]),bins=100, range=[0,1.5])
plt.xscale('log')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF = RandomForestClassifier(n_estimators=150, max_depth = 15, n_jobs=30)
RF2 = RandomForestClassifier(n_estimators=150, max_depth = 15, n_jobs=30)

In [ ]:
RFtrain, RFtest = train_test_split(df_test,test_size=0.5)

In [ ]:
RF.fit(RFtrain[["ITS","TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]], np.ravel(RFtrain[["particle"]]))

In [ ]:
RF.score(RFtest[["ITS","TOF", "TPCROC0", "TPCROC1", "TPCROC2", "TRD","p"]], np.ravel(RFtest[["particle"]]))

In [ ]:
RF2.fit(RFtrain[["ITS_ae","TOF_ae", "TPCROC0_ae", "TPCROC1_ae", "TPCROC2_ae", "TRD_ae","p"]], np.ravel(RFtrain[["particle"]]))

In [ ]:
RF2.score(RFtest[["ITS_ae","TOF_ae", "TPCROC0_ae", "TPCROC1_ae", "TPCROC2_ae", "TRD_ae","p"]], np.ravel(RFtest[["particle"]]))